In [79]:
def displayer(ctrl_seq):
	D = []
	for i in range(10):
		d = []
		for j in range(160):
			d.append(' ')
		D.append(d)

	i, j = 0, 0
	for c in ctrl_seq:
		if c == 0: j += 1
		elif c == 1: j -= 1
		elif c == 2: i += 1
		elif c == 3: i -= 1
		else: D[i][j] = chr(c)

	for i in range(10):
		for j in range(160):
			print(D[i][j], end='')
		print('')

In [82]:
file = open("proto_msgs.crypt", "rb")
byte_cipher = file.read()
print(len(byte_cipher))

commands = byte_cipher.hex()
print(commands)

trial = ""
for i in range(1475):
    num = str(i)
    if len(num) < 4:
        num = ("0"* (4 - len(num))) + num
    trial += num + "01"
byte_trial = bytes.fromhex(trial)

key_candid = bytes(a ^ b for a, b in zip(byte_cipher, byte_trial))

4425
6f6578b288073948fc46264fcf6b1bbb270ccc74dabdf275ce24c266f965b5cd38c25c122f587fe3682ac068133f594711e2442e9af755be6bcd0c0a1a5f0eb5da05172e4b3a8f567a11f4ef1e4561b5da69eb2990dfb747ac1bc58032185cb523b35d04ecc23da4707280bea90b6adc1bf1e52d5d29ebab67a14b0f4623161952590a7f541dc72014fbc06e05373c3d923ae3ebf14ea0812204874e22a164d90073c9aad4bf8acfeb4fcc7b7fc27a8eac94be037c1887301035d5604a34f30574a16b4ab33a96be6e9b9ed6b299fe490cf3002025748fa9d10bb4752d67a7dc71d9d04ff0a35da5d7e992db598fb8f6d2fb215c2b1524a67ad1524fbc2a2e453a6558e4d6076e4adc1e054f966f3be1020c9772fae1d575932c9d38d065eac718a277124e545f81452aa366335b764774f26448abf732ac34a53f0a734b51dfef057c381456c7561709d48127450eaf8519d029e1c3e835911bb69e6d6c63b556930272edc24a862f0afcbed02f4aa661f19e0b7d5593ab1a896b713023693372d9017fd531e7a219fb434025b3333d170abc6de04e27b3028c944eab9544531573429cf4339dcf6677ecf566c2f5b48c04a503ed24a7a20d35465e6aa0ec05e1e14bdcd23a01fc4e03fdd62bddded369f39b6605e8e8a94c4394eb446738965179bb4f51ef7d07bae93195792ba8f677a

In [67]:
with open("key_candid.bin", "wb") as f:
    # Encode the string 'x' to bytes and write it to the file
    f.write(key_candid)

In [83]:
with open("left_check.bin", "wb") as f:
    f.write(bytes(a ^ b for a, b in zip(byte_cipher, byte_trial)))

In [68]:
partial_plaintext = bytes([b1 ^ b2 for b1, b2 in zip(key_candid, byte_cipher)])
with open("partial_plaintext.bin", "wb") as f:
    f.write(partial_plaintext)

In [ ]:
import itertools
import string
from prng import PRNG
from tqdm import tqdm

def is_plausible(plaintext):
    return all(32 <= b <= 126 or b in {0, 1, 2, 3} for b in plaintext)

# Load encrypted file
with open('proto_msgs.crypt', 'rb') as f:
    ciphertext = f.read()

# Use limited charset to make brute-force more feasible (printable ASCII)
charset = string.ascii_letters + string.digits  # You may add punctuation for full set

print("Starting brute force for 8-byte seeds...\n")

for candidate in tqdm(itertools.product(charset, repeat=8)):
    seed = ''.join(candidate).encode()

    prng = PRNG()
    prng.seed(seed)
    keystream = prng.get_random_bytes(len(ciphertext))
    plaintext = bytes([a ^ b for a, b in zip(ciphertext, keystream)])

    if is_plausible(plaintext):
        print(f"\n[+] Plausible seed found: {seed.decode()}")
        ctrl_seq = [plaintext[i + 2] for i in range(0, len(plaintext), 3)]
        display(ctrl_seq)
        break
